# Gemini Speech-to-Text on Kaggle

This notebook transcribes audio files using Google's Gemini API with automatic chunking for long recordings. It processes medical audio with mixed Mandarin Chinese and English content.

In [ ]:
# Install required packages
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install --upgrade tqdm google-generativeai requests pydub

In [ ]:
# Import necessary libraries
import os
import json
import shutil
import datetime
import time
import math
from pathlib import Path
from kaggle_secrets import UserSecretsClient
from google.oauth2 import service_account
from googleapiclient.discovery import build
import google.generativeai as genai
from tqdm import tqdm

# Get secrets
s = UserSecretsClient()

# Google Drive folder IDs
to_be_transcribed = s.get_secret("TO_BE_TRANSCRIBED")
transcribed = s.get_secret("TRANSCRIBED")
PROCESSED_ID = s.get_secret("PROCESSED")

# Gemini API key
GEMINI_API_KEY = s.get_secret("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY not found in Kaggle secrets!")

genai.configure(api_key=GEMINI_API_KEY)

In [ ]:
# Set up Google Drive authentication
sa_json_str = s.get_secret("GDRIVE_SA_JSON")

sa_path = "/kaggle/working/sa_key.json"
with open(sa_path, "w") as f:
    f.write(sa_json_str)

SCOPES = ["https://www.googleapis.com/auth/drive"]
creds = service_account.Credentials.from_service_account_file(sa_path, scopes=SCOPES)
drive_service = build("drive", "v3", credentials=creds)

# Remove the local service account file for security
if os.path.exists(sa_path):
    try:
        os.remove(sa_path)
        print(f"✅ Removal successful: '{sa_path}' has been deleted.")
    except Exception as e:
        print(f"❌ Removal failed: {e}")

# Check for files in the to_be_transcribed folder
def list_files_in_folder(folder_id):
    query = f"'{folder_id}' in parents and trashed=false"
    files = []
    page_token = None
    while True:
        resp = drive_service.files().list(
            q=query,
            spaces="drive",
            fields="nextPageToken, files(id, name)",
            pageToken=page_token
        ).execute()
        files.extend(resp.get("files", []))
        page_token = resp.get("nextPageToken", None)
        if not page_token:
            break
    return files

files = list_files_in_folder(to_be_transcribed)

if not files:
    new_files = False
    print(f"⚠️ No files found in folder ID = {to_be_transcribed!r}. new_files = False.")
else:
    new_files = True
    print(f"✅ Found {len(files)} file(s) in folder ID = {to_be_transcribed!r}. new_files = True.")
    for f in files:
        print(f" • {f['name']} (ID={f['id']})")

In [ ]:
# Download files from Google Drive
import io
from googleapiclient.http import MediaIoBaseDownload

TO_BE_TRANSCRIBED_ID = to_be_transcribed

local_root = "/kaggle/working/from_google_drive"
os.makedirs(local_root, exist_ok=True)

# List all non-folder files
query_files = (
    f"'{TO_BE_TRANSCRIBED_ID}' in parents and "
    "trashed = false and "
    "mimeType != 'application/vnd.google-apps.folder'"
)
response = drive_service.files().list(
    q=query_files,
    spaces="drive",
    fields="files(id, name)"
).execute()
files = response.get("files", [])

if not files:
    print(f"⚠️ No files found in folder ID = {TO_BE_TRANSCRIBED_ID!r}, skip downloading.")
else:
    print(f"🔎 Found {len(files)} file(s) in folder ID = {TO_BE_TRANSCRIBED_ID!r}:")
    for f in files:
        print(f" • {f['name']}  (ID = {f['id']})")

# Download each file
for f in files:
    file_id   = f["id"]
    file_name = f["name"]
    dest_path = os.path.join(local_root, file_name)

    try:
        request = drive_service.files().get_media(fileId=file_id)
        fh = io.FileIO(dest_path, mode="wb")
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        print(f"⬇️  Downloading {file_name!r} …")
        while not done:
            status, done = downloader.next_chunk()
        fh.close()
        print(f"✅ Saved to {dest_path}")
    except Exception as e:
        print(f"❌ Error downloading {file_name!r}: {e}")

In [ ]:
# Audio processing utilities
from pydub import AudioSegment
import subprocess
import tempfile

def get_audio_duration(audio_path):
    """Get audio duration in seconds using pydub."""
    try:
        audio = AudioSegment.from_file(str(audio_path))
        return len(audio) / 1000.0  # Convert milliseconds to seconds
    except Exception as e:
        print(f"Error getting audio duration: {e}")
        return None

def split_audio_into_chunks(audio_path, chunk_duration_seconds=300):
    """Split audio file into chunks of specified duration."""
    chunks = []
    try:
        # Load audio
        audio = AudioSegment.from_file(str(audio_path))
        total_duration = len(audio) / 1000.0  # Convert to seconds
        
        print(f"Total audio duration: {total_duration:.1f} seconds ({total_duration/60:.1f} minutes)")
        
        # Calculate number of chunks
        num_chunks = math.ceil(total_duration / chunk_duration_seconds)
        
        if num_chunks == 1:
            print("Audio is shorter than chunk duration, processing as single file")
            return [audio_path]
        
        print(f"Splitting audio into {num_chunks} chunks of {chunk_duration_seconds} seconds each...")
        
        # Create temporary directory for chunks
        temp_dir = Path(tempfile.mkdtemp())
        
        chunk_duration_ms = chunk_duration_seconds * 1000
        
        for i in range(num_chunks):
            start_ms = i * chunk_duration_ms
            end_ms = min((i + 1) * chunk_duration_ms, len(audio))
            
            # Extract chunk
            chunk = audio[start_ms:end_ms]
            
            # Save chunk
            chunk_path = temp_dir / f"chunk_{i+1:03d}.wav"
            chunk.export(chunk_path, format="wav")
            
            print(f"Created chunk {i+1}/{num_chunks} (duration: {(end_ms-start_ms)/1000:.1f}s)")
            chunks.append(chunk_path)
        
        print(f"Successfully created {len(chunks)} chunks")
        return chunks
        
    except Exception as e:
        print(f"Error splitting audio: {e}")
        return [audio_path]

In [ ]:
# Gemini transcription functions
def transcribe_audio_chunk(chunk_path, chunk_number, total_chunks):
    """Transcribe a single audio chunk using Gemini API."""
    try:
        print(f"Transcribing chunk {chunk_number}/{total_chunks}...")
        
        # Upload audio file to Gemini
        audio_file = genai.upload_file(path=str(chunk_path), display_name=f"chunk_{chunk_number}")
        
        # Wait for file to be processed
        while audio_file.state.name == "PROCESSING":
            time.sleep(1)
            audio_file = genai.get_file(audio_file.name)
        
        # Prepare the prompt
        prompt = f"""You are transcribing chunk {chunk_number} of {total_chunks} from a medical audio recording that contains both Mandarin Chinese and Medical English terminology.

CRITICAL INSTRUCTIONS:
1. Transcribe the ENTIRE audio chunk from start to finish without stopping
2. Include ALL content, even if there are pauses or quiet sections
3. Do NOT summarize or skip any portions of the audio
4. Continue transcribing until you reach the absolute end of this audio chunk
5. This is chunk {chunk_number} of {total_chunks} - transcribe ONLY what you hear in this specific chunk

LANGUAGE AND CONTENT:
- The audio contains mixed Mandarin Chinese (Traditional Chinese zh-tw) and English medical terminology
- Speakers may switch between languages mid-sentence
- Medical terms may be spoken in English even within Chinese sentences
- Include all medical terminology, abbreviations, and technical language exactly as spoken

FORMAT:
- Transcribe verbatim what is said
- When speakers switch languages, transcribe in the language being spoken
- Preserve medical terms in their original language (usually English)
- Include speaker changes if distinguishable
- Do NOT add any chunk markers or headers - just transcribe the content

IMPORTANT: This is a complete transcription of chunk {chunk_number}. Transcribe every single word from the beginning to the end of this audio chunk."""
        
        # Generate transcription
        model = genai.GenerativeModel('gemini-2.5-pro')
        response = model.generate_content([prompt, audio_file])
        
        # Delete uploaded file
        genai.delete_file(audio_file.name)
        
        return response.text
        
    except Exception as e:
        print(f"Error transcribing chunk {chunk_number}: {e}")
        return None

def transcribe_audio_file(audio_path, chunk_duration_seconds=300):
    """Transcribe an audio file by splitting it into chunks."""
    try:
        # Split audio into chunks
        chunks = split_audio_into_chunks(audio_path, chunk_duration_seconds)
        
        if not chunks:
            print("No chunks created, cannot proceed with transcription")
            return None
        
        # Transcribe each chunk
        transcriptions = []
        total_chunks = len(chunks)
        
        for i, chunk_path in enumerate(chunks, 1):
            print(f"\n--- Processing chunk {i}/{total_chunks} ---")
            chunk_transcript = transcribe_audio_chunk(chunk_path, i, total_chunks)
            
            if chunk_transcript:
                transcriptions.append(chunk_transcript)
                print(f"Successfully transcribed chunk {i}/{total_chunks}")
            else:
                print(f"Failed to transcribe chunk {i}/{total_chunks}")
        
        # Clean up chunk files if they were created
        if len(chunks) > 1:  # Only clean up if we created temporary chunks
            temp_dir = chunks[0].parent
            for chunk in chunks:
                if chunk.exists():
                    chunk.unlink()
            if temp_dir.exists() and temp_dir != audio_path.parent:
                temp_dir.rmdir()
        
        # Merge all transcriptions
        if transcriptions:
            print(f"\nMerging {len(transcriptions)} chunk transcriptions...")
            
            # Join chunks with timestamps
            merged_transcript = ""
            for i, transcript in enumerate(transcriptions, 1):
                timestamp = f"[{(i-1)*chunk_duration_seconds//60:02d}:{(i-1)*chunk_duration_seconds%60:02d}:00.000]"
                merged_transcript += f"{timestamp}\n{transcript.strip()}\n\n"
            
            return merged_transcript.strip()
        else:
            print("No chunks were successfully transcribed")
            return None
            
    except Exception as e:
        print(f"Error in chunked transcription: {e}")
        return None

In [ ]:
# Main transcription process
AUDIO_EXT = {'.wav', '.mp3', '.m4a', '.flac', '.ogg', '.webm'}
INBOX_DIR = Path("/kaggle/working/from_google_drive")
TRANSCRIPTS_DIR = Path("/kaggle/working/transcription")
CHUNK_DURATION_SECONDS = 300  # 5 minutes

# Ensure directories exist
TRANSCRIPTS_DIR.mkdir(parents=True, exist_ok=True)

# Gather audio files
audio_files = [p for p in INBOX_DIR.rglob("*") if p.suffix.lower() in AUDIO_EXT]
total = len(audio_files)

if total == 0:
    print(f"[{datetime.datetime.now():%Y-%m-%d %H:%M:%S}] 📂 No audio files found under {INBOX_DIR}")
    print("Please place your audio files in /kaggle/working/from_google_drive and re-run.")
    new_files = False
else:
    new_files = True

# Transcription process
if not new_files:
    print(f"[{datetime.datetime.now():%Y-%m-%d %H:%M:%S}] ⚠️ new_files == False → Skipping transcription.")
else:
    print(f"[{datetime.datetime.now():%Y-%m-%d %H:%M:%S}] 🎧 Found {total} audio file(s) under {INBOX_DIR}")
    
    # Transcription loop with progress bar
    with tqdm(audio_files, desc="Transcribing files", unit="file") as pbar:
        for audio in pbar:
            pbar.set_description(f"Transcribing: {audio.name}")
            
            # Transcribe using Gemini with chunking
            transcript = transcribe_audio_file(audio, CHUNK_DURATION_SECONDS)
            
            if transcript:
                # Save transcript
                out_txt = TRANSCRIPTS_DIR / f"{audio.stem}.txt"
                with open(out_txt, "w", encoding="utf-8") as f:
                    f.write(transcript)
                print(f"\n✅ Saved transcript to: {out_txt}")
            else:
                print(f"\n❌ Failed to transcribe {audio.name}")
    
    print(f"\n[{datetime.datetime.now():%Y-%m-%d %H:%M:%S}] 🎉 All transcription jobs finished! Transcripts are in {TRANSCRIPTS_DIR}")

In [ ]:
# Parse transcripts into 5-minute blocks (similar to Whisper version)
import re
from datetime import timedelta

# Simple parser for Gemini output
def parse_transcript_simple(text: str) -> str:
    """Parse transcript into 5-minute blocks with preserved formatting."""
    lines = text.strip().split('\n')
    
    result = []
    current_block = []
    
    for line in lines:
        line = line.strip()
        if line.startswith('[') and line.endswith(']'):
            # This is a timestamp line
            if current_block:
                result.append('\n'.join(current_block))
                result.append('')  # blank line
                current_block = []
            result.append(line)
        elif line:
            current_block.append(line)
    
    if current_block:
        result.append('\n'.join(current_block))
    
    return '\n'.join(result)

# Process transcripts
PARSED_DIR = Path("/kaggle/working/parsed")
PARSED_DIR.mkdir(parents=True, exist_ok=True)

txt_files = list(TRANSCRIPTS_DIR.glob("*.txt"))
if not txt_files:
    print(f"[{datetime.datetime.now():%Y-%m-%d %H:%M:%S}] ⚠️ No .txt files found in {TRANSCRIPTS_DIR}. Skipping parsing.")
else:
    for txtfile in txt_files:
        with txtfile.open(encoding="utf-8") as f:
            text = f.read()

        processed = parse_transcript_simple(text)
        out_path = PARSED_DIR / txtfile.name.replace(".txt", "_parsed.txt")

        with out_path.open("w", encoding="utf-8") as f:
            f.write(processed)
        print(f"🔧 Processed {txtfile.name} → {out_path.name}")

    print("\n✅ All transcripts parsed.")

In [ ]:
# Get summary prompt from Google Doc (same as Whisper version)
from googleapiclient.discovery import build

DOC_ID = '1p44XUpBu7lPjyux4eANd_9FHT5F1UDbgUyx7q6Libvk'

def get_doc_text(doc_id: str, creds) -> str:
    service = build('docs', 'v1', credentials=creds)
    doc = service.documents().get(documentId=doc_id).execute()
    text = []
    for element in doc.get('body', {}).get('content', []):
        if 'paragraph' in element:
            for run in element['paragraph'].get('elements', []):
                txt = run.get('textRun', {}).get('content')
                if txt:
                    text.append(txt)
    return ''.join(text).strip()

# Retrieve the system prompt from the Google Doc
SYSTEM_PROMPT = get_doc_text(DOC_ID, creds)
print("[INFO] SYSTEM_PROMPT loaded from Google Doc. Preview:\n", SYSTEM_PROMPT[:200] + "...")

In [ ]:
# Generate summaries using Gemini (same as Whisper version)
def generate_summary_with_gemini(speech_text: str, system_prompt: str) -> str:
    model = genai.GenerativeModel("gemini-2.5-pro")
    full_prompt = system_prompt.strip() + "\n\n" + speech_text.strip()
    try:
        response = model.generate_content(
            full_prompt,
            generation_config=genai.types.GenerationConfig(temperature=0.5),
            stream=False,
        )
        return response.text
    except Exception as e:
        print(f"[ERROR] Gemini API error: {e}")
        return None

def process_all_txt_files(parsed_dir: Path, markdown_dir: Path, system_prompt: str):
    parsed_dir = Path(parsed_dir)
    markdown_dir = Path(markdown_dir)
    markdown_dir.mkdir(parents=True, exist_ok=True)

    txt_files = list(parsed_dir.glob("*.txt"))
    if not txt_files:
        print(f"[INFO] No parsed .txt files found in {parsed_dir}. Skipping summarization.")
        return

    print(f"[INFO] Found {len(txt_files)} .txt files in {parsed_dir}")

    for txt_path in txt_files:
        print(f"\n[INFO] Processing: {txt_path.name}")
        try:
            with open(txt_path, "r", encoding="utf-8") as f:
                speech_text = f.read().strip()
        except Exception as e:
            print(f"[ERROR] Could not read {txt_path}: {e}")
            continue

        if not speech_text:
            print(f"[WARNING] {txt_path.name} is empty, skipping.")
            continue

        summary_md = generate_summary_with_gemini(speech_text, system_prompt)
        if summary_md is None:
            print(f"[ERROR] Gemini API failed for {txt_path.name}, skipping.")
            continue

        md_path = markdown_dir / (txt_path.stem.replace("_parsed", "") + ".md")

        try:
            with open(md_path, "w", encoding="utf-8") as f:
                f.write(summary_md)
            print(f"[INFO] Saved summary → {md_path.name}")
        except Exception as e:
            print(f"[ERROR] Could not save {md_path}: {e}")

# Set directories
PARSED_DIR = Path("/kaggle/working/parsed")
MARKDOWN_DIR = Path("/kaggle/working/markdown")

# Only run if there are files to process
if list(PARSED_DIR.glob("*.txt")):
    process_all_txt_files(PARSED_DIR, MARKDOWN_DIR, SYSTEM_PROMPT)
    print("\n✅ All summaries generated.")
else:
    print(f"[INFO] No .txt files found in {PARSED_DIR}, nothing to summarize.")

In [ ]:
# Upload to HackMD (same as Whisper version)
import requests

hackmd_token = s.get_secret("HACKMD_TOKEN")
if hackmd_token is None:
    raise ValueError("HACKMD_TOKEN not found in Kaggle secrets!")

def upload_to_hackmd(md_content: str, filename: str, api_token: str) -> dict:
    """Uploads a single markdown string to HackMD."""
    # Derive a clean title from the filename
    if filename.endswith('.md'):
        filename = filename[:-3]
    raw_title = filename.replace('_parsed', '').strip()
    title = raw_title.replace('_', ' ').strip()

    # Ensure there's a top-level heading
    md_lines = md_content.lstrip().splitlines()
    if not md_lines or not md_lines[0].strip().startswith("# "):
        md_content = f"# {title}\n\n" + md_content.lstrip()
    else:
        md_lines[0] = f"# {title}"
        md_content = "\n".join(md_lines)

    # Append hashtag
    hashtag = "#gemini-stt-project"
    content_lines = md_content.rstrip().splitlines()
    if not any(line.strip() == hashtag for line in content_lines[-3:]):
        md_content = md_content.rstrip() + "\n\n" + hashtag + "\n"

    url = "https://api.hackmd.io/v1/notes"
    headers = {
        "Authorization": f"Bearer {api_token}",
        "Content-Type": "application/json"
    }
    data = {
        "title": title,
        "content": md_content,
        "readPermission": "guest",
        "writePermission": "signed_in"
    }
    response = requests.post(url, headers=headers, json=data)
    if response.ok:
        note_id = response.json().get("id")
        shared_url = f"https://hackmd.io/{note_id}"
        print(f"[INFO] Uploaded to HackMD: {shared_url}")
        return {"title": title, "url": shared_url}
    else:
        print(f"[ERROR] HackMD upload failed for {filename}: {response.status_code} {response.text}")
        return None

def batch_upload_markdown_and_move(markdown_dir: Path, uploaded_dir: Path, hackmd_token: str) -> list:
    markdown_dir = Path(markdown_dir)
    uploaded_dir = Path(uploaded_dir)
    uploaded_dir.mkdir(parents=True, exist_ok=True)

    md_files = list(markdown_dir.glob("*.md"))
    if not md_files:
        print(f"[INFO] No markdown files found in {markdown_dir}, skipping upload.")
        return []

    print(f"[INFO] Found {len(md_files)} markdown files to upload.")

    shared_links = []
    for md_file in md_files:
        print(f"[INFO] Processing: {md_file.name}")
        try:
            with open(md_file, "r", encoding="utf-8") as f:
                md_content = f.read()
        except Exception as e:
            print(f"[ERROR] Could not read {md_file.name}: {e}")
            continue

        result = upload_to_hackmd(md_content, md_file.name, hackmd_token)
        if result:
            shared_links.append(result)
            dest_file = uploaded_dir / md_file.name
            try:
                shutil.move(str(md_file), dest_file)
                print(f"[INFO] Moved {md_file.name} → {dest_file}")
            except Exception as e:
                print(f"[ERROR] Failed to move {md_file.name}: {e}")
    return shared_links

# Upload to HackMD
MARKDOWN_DIR = Path("/kaggle/working/markdown")
UPLOADED_DIR = Path("/kaggle/working/uploaded")

if list(MARKDOWN_DIR.glob("*.md")):
    shared_links = batch_upload_markdown_and_move(MARKDOWN_DIR, UPLOADED_DIR, hackmd_token)
    print("\n✅ All markdown files uploaded to HackMD.")
else:
    print(f"[INFO] No .md files found in {MARKDOWN_DIR}, nothing to upload.")

In [ ]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from pathlib import Path
import datetime

# Replace with your secret retrieval method
INBOX_ID = s.get_secret("TO_BE_TRANSCRIBED")
ARCHIVE_ID = s.get_secret("TRANSCRIBED")
PROCESSED_ID = s.get_secret("PROCESSED")

# Local paths
WORKING = Path("/kaggle/working")
TRANS_DIR = WORKING / "transcription"
PARSED_DIR = WORKING / "parsed"
UPLOADED_MD = WORKING / "uploaded"
AUDIO_LOCAL = WORKING / "from_google_drive"

# Build drive service
drive = build("drive", "v3", credentials=creds)

def log(msg): 
    print(f"[{datetime.datetime.now():%H:%M:%S}] {msg}")

def ensure_subfolder(parent_id: str, name: str) -> str:
    """Return id of subfolder 'name' under parent, creating if absent."""
    q = (f"'{parent_id}' in parents and mimeType='application/vnd.google-apps.folder' "
         f"and name='{name}' and trashed=false")
    res = drive.files().list(
        q=q,
        spaces="drive",
        fields="files(id)",
        supportsAllDrives=True
    ).execute()
    if res["files"]:
        return res["files"][0]["id"]
    meta = {
        "name": name,
        "mimeType": "application/vnd.google-apps.folder",
        "parents": [parent_id]
    }
    return drive.files().create(
        body=meta,
        fields="id",
        supportsAllDrives=True
    ).execute()["id"]

def upload_file(local: Path, parent_id: str):
    media = MediaFileUpload(local, resumable=False)
    meta  = {"name": local.name, "parents": [parent_id]}
    drive.files().create(
        body=meta,
        media_body=media,
        fields="id",
        supportsAllDrives=True
    ).execute()
    log(f"  ↳ uploaded {local.name}")

def move_audio(audio_name: str):
    """Move audio file from inbox to archive."""
    q = f"'{INBOX_ID}' in parents and name='{audio_name}' and trashed=false"
    res = drive.files().list(
        q=q,
        spaces="drive",
        fields="files(id)",
        supportsAllDrives=True
    ).execute().get("files", [])
    
    if not res:
        return
    fid = res[0]["id"]
    drive.files().update(
        fileId=fid,
        addParents=ARCHIVE_ID,
        removeParents=INBOX_ID,
        fields="id",
        supportsAllDrives=True
    ).execute()
    log(f"  ↳ moved {audio_name} → transcribed")

# Process markdown files
md_files = list(UPLOADED_MD.glob("*.md"))
if not md_files:
    log("ℹ️  No markdown files in /uploaded – nothing to sync.")
else:
    for md in md_files:
        stem = md.stem
        folder_id = ensure_subfolder(PROCESSED_ID, stem)
        log(f"📂 Drive subfolder '{stem}' (id {folder_id})")

        txt_path    = TRANS_DIR  / f"{stem}.txt"
        parsed_path = PARSED_DIR / f"{stem}_parsed.txt"

        for p in (txt_path, parsed_path, md):
            if p.exists():
                upload_file(p, folder_id)

        # Move corresponding audio
        for audio_local in AUDIO_LOCAL.glob(f"{stem}.*"):
            if audio_local.is_file():
                move_audio(audio_local.name)
                break

        # Verify contents
        present = {f["name"] for f in drive.files().list(
            q=f"'{folder_id}' in parents and trashed=false",
            spaces="drive",
            fields="files(name)",
            supportsAllDrives=True
        ).execute()["files"]}
        expected = {txt_path.name, parsed_path.name, md.name}
        if expected - present:
            log(f"  ✖ missing {expected - present}")
        else:
            log("  ✅ files verified")

        md.unlink(missing_ok=True)

# Move any remaining audio files
for audio_local in AUDIO_LOCAL.glob("*"):
    if audio_local.is_file():
        move_audio(audio_local.name)


In [ ]:
# Email HackMD links (same as Whisper version)
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.header import Header

# Skip if no links were produced
if not (globals().get("shared_links") and shared_links):
    print("[INFO] No uploaded Markdown links – skipping email step.")
else:
    # Retrieve email secrets
    email_user = s.get_secret("EMAIL_USER")
    email_pass = s.get_secret("EMAIL_PASS")
    email_to   = s.get_secret("EMAIL_TO")

    if not all([email_user, email_pass, email_to]):
        print("[WARN] Email secrets missing – email not sent.")
    else:
        # Build email body
        subject = "📝 Your Uploaded HackMD Speech Summaries (Gemini STT)"
        body_lines = [
            "Hello,",
            "",
            "Your audio files were transcribed using Gemini STT with chunking",
            "and summarized using Gemini 2.5 Pro. The summaries are now",
            "available on HackMD:",
            ""
        ] + [f"- {link['title']}: {link['url']}" for link in shared_links] + [
            "",
            "If you have questions just reply to this email.",
            "",
            "Best regards,",
            "Gemini-STT Bot"
        ]
        body = "\n".join(body_lines)

        # Compose & send
        msg             = MIMEMultipart()
        msg["From"]     = email_user
        msg["To"]       = email_to
        msg["Subject"]  = Header(subject, "utf-8")
        msg.attach(MIMEText(body, "plain", "utf-8"))

        try:
            with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
                server.login(email_user, email_pass)
                server.send_message(msg)
            print("[INFO] Email sent successfully.")
        except Exception as e:
            print(f"[ERROR] Email send failed: {e}")